In [33]:
using GLMakie
using Revise
include("EFITModule/EFIT.jl")


Main.EFIT

# Initialize files

In [46]:
paths = Vector{Tuple{Int64,String}}()
dir = "/media/data/2022 Data/RailTest_2,5MHz_3_May/"
for f in readdir(dir)
    fname, ftype = split(f,".")
    i = split(fname,"-")[end]
    if ftype == "bov"
        push!(paths,(parse(Int,i),"$dir/$f"))
    end
end
sort!(paths)
bovs = [i[2] for i in paths];
data = Main.EFIT.readBOV(bovs[1]);
print(size(data))


(434, 358, 716)

# Volumetric animation

In [47]:
data = Main.EFIT.readBOV(bovs[1]);
fig,ax,plt = volume(data,algorithm=:mip,colorrange = (0, 0.01),colormap=:curl, transparency=true)
record(fig, "Animation_Volume.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    plt.volume = data
end

"Animation_Volume.mp4"

# 2D Slices

## X Slice

In [74]:
data = Main.EFIT.readBOV(bovs[1]);
fig,ax,plt = heatmap(data[floor(Int,end/2),:,:],colorrange = (0, 0.01),colormap=:curl)
record(fig, "Animation_XSlice.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    plt.heatmap = data[floor(Int,end/2),:,:]
end

ErrorException: Screen not open!

In [73]:
data = Main.EFIT.readBOV(bovs[1]);
fig,ax,plt = heatmap(data[:,floor(Int,end/2),:],colorrange = (0, 0.01),colormap=:curl)
record(fig, "Animation_YSlice.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    plt.heatmap = data[:,floor(Int,end/2),:]
end

ErrorException: Screen not open!

# Composite

In [78]:
data = Main.EFIT.readBOV(bovs[1]);

fig = Figure()

vscene = LScene(fig[1, 1])
vplt = volume!(vscene, data,algorithm=:mip,colorrange = (0, 0.01),colormap=:curl, transparency=true)

fig[1,2] = LScene(fig[1, 2])
xplt = heatmap!(fig[1,2],data[floor(Int,end/2),:,:],colorrange = crange,colormap=cmap)

fig[2,1] = Axis(fig, title = "Y Slice")
yplt = heatmap!(fig[2,1],data[:,floor(Int,end/2),:],colorrange = crange,colormap=cmap)

fig[2,2] = Axis(fig, title = "Z Slice")
zplt = heatmap!(fig[2,2],data[:,:,floor(Int,end/2)],colorrange = crange,colormap=cmap)

crange =(0, 0.01)
cmap=:curl 

record(fig, "Animation_Composite.mp4", bovs; framerate = 30) do path
    Main.EFIT.readBOV!(data, path)
    vplt.volume = data
    xplt.heatmap = data[floor(Int,end/2),:,:]
    yplt.heatmap = data[:,floor(Int,end/2),:]
    zplt.heatmap = data[:,:,floor(Int,end/2)]


end

"Animation_Composite.mp4"